<a href="https://colab.research.google.com/github/NAVEED261/MY-AI-ASSISTANT/blob/main/Text_Analysis_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain langgraph


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.0/378.0 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: msgpack
    Found exi

In [3]:
!git clone https://github.com/NirDiamant/GenAI_Agents


Cloning into 'GenAI_Agents'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 144 (delta 59), reused 109 (delta 30), pack-reused 0 (from 0)
Receiving objects: 100% (144/144), 16.36 MiB | 17.61 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [12]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Introduction to LangGraph\n",
    "\n",
    "LangGraph is a framework for creating applications using graph-based workflows. Each node represents a function or computational step, and edges define the flow between these nodes based on certain conditions.\n",
    "\n",
    "## Key Features:\n",
    "- State Management\n",
    "- Flexible Routing\n",
    "- Persistence\n",
    "- Visualization"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Tutorial Overview: Text Analysis Pipeline\n",
    "\n",
    "In this tutorial, we'll demonstrate the power of LangGraph by building a multi-step text analysis pipeline. Our use case will focus on processing a given text through three key stages:\n",
    "\n",
    "1. **Text Classification**: We'll categorize the input text into predefined categories (e.g., News, Blog, Research, or Other).\n",
    "2. **Entity Extraction**: We'll identify and extract key entities such as persons, organizations, and locations from the text.\n",
    "3. **Text Summarization**: Finally, we'll generate a concise summary of the input text.\n",
    "\n",
    "This pipeline showcases how LangGraph can be used to create a modular, extensible workflow for natural language processing tasks. By the end of this tutorial, you'll understand how to construct a graph-based application that can be easily modified or expanded for various text analysis needs."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Import Required Libraries\n",
    "This cell imports all the necessary modules and classes for our LangGraph tutorial."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 28,
   "metadata": {},
   "outputs": [],
   "source": [
    "import os\n",
    "from typing import TypedDict, List\n",
    "from langgraph.graph import StateGraph, END\n",
    "from langchain.prompts import PromptTemplate\n",
    "from langchain_openai import ChatOpenAI\n",
    "from langchain.schema import HumanMessage\n",
    "from langchain_core.runnables.graph import MermaidDrawMethod\n",
    "from IPython.display import display, Image\n",
    "\n",
    "from dotenv import load_dotenv"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Set Up API Key\n",
    "This cell loads environment variables and sets up the OpenAI API key. Make sure you have a `.env` file with your `OPENAI_API_KEY`."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load environment variables\n",
    "load_dotenv()\n",
    "\n",
    "# Set OpenAI API key\n",
    "os.environ[\"OPENAI_API_KEY\"] = os.getenv('OPENAI_API_KEY')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Building the Text Processing Pipeline\n",
    "\n",
    "### Define State and Initialize LLM\n",
    "Here we define the State class to hold our workflow data and initialize the ChatOpenAI model."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [],
   "source": [
    "class State(TypedDict):\n",
    "    text: str\n",
    "    classification: str\n",
    "    entities: List[str]\n",
    "    summary: str\n",
    "\n",
    "llm = ChatOpenAI(model=\"gpt-4o-mini\", temperature=0)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Define Node Functions\n",
    "These functions define the operations performed at each node of our graph: classification, entity extraction, and summarization."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 20,
   "metadata": {},
   "outputs": [],
   "source": [
    "def classification_node(state: State):\n",
    "    ''' Classify the text into one of the categories: News, Blog, Research, or Other '''\n",
    "    prompt = PromptTemplate(\n",
    "        input_variables=[\"text\"],\n",
    "        template=\"Classify the following text into one of the categories: News, Blog, Research, or Other.\\n\\nText:{text}\\n\\nCategory:\"\n",
    "    )\n",
    "    message = HumanMessage(content=prompt.format(text=state[\"text\"]))\n",
    "    classification = llm.predict_messages([message]).content.strip()\n",
    "    return {\"classification\": classification}\n",
    "\n",
    "\n",
    "def entity_extraction_node(state: State):\n",
    "    ''' Extract all the entities (Person, Organization, Location) from the text '''\n",
    "    prompt = PromptTemplate(\n",
    "        input_variables=[\"text\"],\n",
    "        template=\"Extract all the entities (Person, Organization, Location) from the following text. Provide the result as a comma-separated list.\\n\\nText:{text}\\n\\nEntities:\"\n",
    "    )\n",
    "    message = HumanMessage(content=prompt.format(text=state[\"text\"]))\n",
    "    entities = llm.predict_messages([message]).content.strip().split(\", \")\n",
    "    return {\"entities\": entities}\n",
    "\n",
    "\n",
    "def summarization_node(state: State):\n",
    "    ''' Summarize the text in one short sentence '''\n",
    "    prompt = PromptTemplate(\n",
    "        input_variables=[\"text\"],\n",
    "        template=\"Summarize the following text in one short sentence.\\n\\nText:{text}\\n\\nSummary:\"\n",
    "    )\n",
    "    message = HumanMessage(content=prompt.format(text=state[\"text\"]))\n",
    "    summary = llm.predict_messages([message]).content.strip()\n",
    "    return {\"summary\": summary}"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Create Tools and Build Workflow\n",
    "This cell builds the StateGraph workflow."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "metadata": {},
   "outputs": [],
   "source": [
    "workflow = StateGraph(State)\n",
    "\n",
    "# Add nodes to the graph\n",
    "workflow.add_node(\"classification_node\", classification_node)\n",
    "workflow.add_node(\"entity_extraction\", entity_extraction_node)\n",
    "workflow.add_node(\"summarization\", summarization_node)\n",
    "\n",
    "# Add edges to the graph\n",
    "workflow.set_entry_point(\"classification_node\") # Set the entry point of the graph\n",
    "workflow.add_edge(\"classification_node\", \"entity_extraction\")\n",
    "workflow.add_edge(\"entity_extraction\", \"summarization\")\n",
    "workflow.add_edge(\"summarization\", END)\n",
    "\n",
    "# Compile the graph\n",
    "app = workflow.compile()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Visualizing the Workflow\n",
    "This cell creates a visual representation of our workflow using Mermaid"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 29,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/jpeg": "/9j/4AAQSkZJRgABAQAAAQABAAD/4gHYSUNDX1BST0ZJTEUAAQEAAAHIAAAAAAQwAABtbnRyUkdCIFhZWiAH4AABAAEAAAAAAABhY3NwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAQAA9tYAAQAAAADTLQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAlkZXNjAAAA8AAAACRyWFlaAAABFAAAABRnWFlaAAABKAAAABRiWFlaAAABPAAAABR3dHB0AAABUAAAABRyVFJDAAABZAAAAChnVFJDAAABZAAAAChiVFJDAAABZAAAAChjcHJ0AAABjAAAADxtbHVjAAAAAAAAAAEAAAAMZW5VUwAAAAgAAAAcAHMAUgBHAEJYWVogAAAAAAAAb6IAADj1AAADkFhZWiAAAAAAAABimQAAt4UAABjaWFlaIAAAAAAAACSgAAAPhAAAts9YWVogAAAAAAAA9tYAAQAAAADTLXBhcmEAAAAAAAQAAAACZmYAAPKnAAANWQAAE9AAAApbAAAAAAAAAABtbHVjAAAAAAAAAAEAAAAMZW5VUwAAACAAAAAcAEcAbwBvAGcAbABlACAASQBuAGMALgAgADIAMAAxADb/2wBDAAMCAgMCAgMDAwMEAwMEBQgFBQQEBQoHBwYIDAoMDAsKCwsNDhIQDQ4RDgsLEBYQERMUFRUVDA8XGBYUGBIUFRT/2wBDAQMEBAUEBQkFBQkUDQsNFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBT/wAARCAGwALcDASIAAhEBAxEB/8QAHQABAQACAwEBAQAAAAAAAAAAAAYFBwMECAIBCf/EAFQQAAEDAwEDBQoKBQoDCAMAAAEAAgMEBQYRBxIhExUxVpQIFBYXIkFRVdHTMjZUYXF0lbLS1CM0coGTMzVCUmJzdZKxsyQlgxgnR1NXY5GhgpbB/8QAGwEBAQADAQEBAAAAAAAAAAAAAAECAwUEBgf/xAA0EQEAAQICBQoFBQEBAAAAAAAAAQIRA1ESFCExkQQTM0FSYnGSsdEFFWGhwSNTgeHwIjL/2gAMAwEAAhEDEQA/AP6poiICIiAiIgLrVdxpLeAaqqhpgegzSBn+qwL6iszCaaKiqZbbZY3GN1bBoJqtwOjhESDuRjiN/wCE467u6AHO7FLgWO0ji9tlo5Zid509REJpXH0l79XH95Xo0KKOknblH5/0/VbZu34VWX1xQdpZ7U8KrL64oO0s9qeCtl9T0HZmexPBWy+p6DszPYn6P1+y7Dwqsvrig7Sz2p4VWX1xQdpZ7U8FbL6noOzM9ieCtl9T0HZmexP0fr9jYeFVl9cUHaWe1PCqy+uKDtLPangrZfU9B2ZnsTwVsvqeg7Mz2J+j9fsbDwpsvreg7Sz2rvU1ZBWx79PPHOz+tE8OH/yF0fBay+qKDszPYujUbP8AH5X8rDbIbfVDXdqre3vaZpP9tmhP0HUfMlsGeuY4f0mxRIp2huFbY6+C2XaZ1ZBOd2kubmtaXu0/kpg0BoeeJDmgNdoRo0gB1EtVdE0STFhERYIIiICIiAiIgIiICn86rpqTHnxU0pgqq2aGhilBILDLI2MuGnnaHE/uVApjaE3krLS1p13KCvpaqTQa6Rtlbvn9zS4/uW/AiJxaYnNY3qChooLbRU9JSxNgpqeNsUUTBo1jGjRoHzAABc6ItMzMzeUFD53tswzZpdaS2ZFeTR3CqhNTHTQ0k9S9sIdumV4iY7k2b3Dffo3XXjwVwvOfdLsuFnyekyDDLRl7do8FrMFtuVjthrLdWMMpcKGt11a1m8N7edubofvB+vBQWtq7oO1XDbjf9nMlDXw1FuhpDDWMoKqRk0solc9r3CHcia0Rt0e5+68ucAdWkLK2jb9gV9zXwSpL9rfjNNTMp5qOeFk0sWvKRxyvjEcjm7rtWscTwPoUTaau8YV3R96r7vjV3qKTLbNZ6aGvtNE+qpKapgfUNmZPI0HkmjlmuDnaAtB48NFp8W/M8hyHA7nkdmz+45fa80irL2ZIJxZaGlEk0TDSxNPJyMDZIzykbXuDeUL3DiEHoWr7pTBWsvzLdcKu71tl78ZVwUVrrJRFNTb4kie9sJax2rHAanyhxbvAjXKbENrtDtp2f2rIqWlqqGeopYJaqmnpJ4WRSvja8sjfLGwTNG9oJGatOnAqQ2JYdcqTZxtHt1RbZrZW3XJsglibVwuhMzZamURS8QCWubukO6CNNOCyHcu3irdsixzGrlj17x+7Y1aqO2Vkd3oH07JJY4+TcYXnyZW6x67zCRo5vpQbeREQY3I7O2/2SroS4MkkaHRSHX9FK0h0cg087Xta4fOAvjFbwchxq1XMtDH1dNHM5o6GuLQSP3HULvV9bFbaGoq53bsFPG6WR3oa0Ek//AWHwCgltmFWSnnaWTtpY3SNI0LXkbzhp8xJW+NuDN89nCb+kL1M+iItCCIiAiIgIiICIiAuKppoq2mlp542zQSsMckbxq1zSNCCPQQuVE3bYEvarkcWdDZrtKWwt0ioLjK7yJ2cA2ORx6Jh0aH4Y0c3Ul7WdDIth2zzLrzUXa94RYLvdKnd5asrbdFLLJutDW7znNJOjWgfQArGrpIK+mkp6mGOop5Wlr4pWBzHj0EHgQp44BSQH/l9yu1qZrryVLXPdGPoZJvNaPmAA+ZeiZw8TbVNp+39fdlslPHubdlDg0HZvixDRoAbTBwHT/V+cqvxXDbDg1r5tx2z0Njt/KGXvW307YY986au3WgDU6Dj8y6HgTUdar9/Gh90ngTUdar9/Gh90nN4fb+0paM1Qil/Amo61X7+ND7pSlst12q9qeR4/JlN45uoLPbK6Etlh5XlZ5q5km9+j+DpTRacBx3uJ8zm8Pt/aS0ZtpqezDZ7jG0Gnp4Mnx+25BDTOL4Y7lSsnbG4jQlocDoSFweBNR1qv38aH3SeBNR1qv38aH3Sc3h9v7SWjNPf9mvZP/6b4t9kQfhWcxPZRhGz2snr8bxWy49VSxGKWpt9FHTvdHqHFrnNA4agHT5lyjCagH4034/9aH3S/Y9n1ulcDcaivvQHER3GrfJF++IEMP72lNDDjfXwj3sWjNx1MzM8kZS0ukmPRva+pqx8Csc0hzYoj/SZqPLf8Egbg3tX7lWvljGxtDWgNaBoABoAF9LXXXpWiNkQXERFrQREQEREBERAREQEREBERAREQFr+xlvj7zMAnf8ABux6jzad83XTz/T5h9J82wFr+x6+PrM+LdPBuycAG736zdenz6fTw6dPOg2AiIgIiICIiAiIgIiICIiAiIgIiICIiAiIgIiIC17YgPH9mp32k+DVj8kDyh/xV24k6dB+nzHo8+wlr2xaeP8AzXid7waseo3Rpp31dvP5/Pw9qDYSIiAiIgIiICIiAiIgIiICL8c4MaXOIa0DUkngAoo5he7sBUWW2UJtr+MNRcKl8ckzfM8RtjO609I1OpHSAt2HhVYt9H2W11siiOfcw+QWPtc3u059zD5BY+1ze7W7Va844wWW6KI59zD5BY+1ze7Tn3MPkFj7XN7tNVrzjjBZboojn3MPkFj7XN7tOfcw+QWPtc3u01WvOOMFluiiOfcw+QWPtc3u059zD5BY+1ze7TVa844wWV1ylqYLdVS0VOyrrGRPdBTyS8k2WQA7rS/Q7oJ0Guh0110K8KbMe7yq8v7pXmXxZ1dHc7+63Y9PCbmHPoe96iqMszhyALg1tS4lpI05I8RvFevefcw+QWPtc3u1qDGe5+mxbb9f9q9Lb7MbxdafkxSGeQRU8rtBNMw8lrvSADX9p/8AW4NVrzjjBZ6WRRHPuYfILH2ub3ac+5h8gsfa5vdpqteccYLLdFEc+5h8gsfa5vdpz7mHyCx9rm92mq15xxgst0URz7mHyCx9rm92nPuYfILH2ub3aarXnHGCy3RRHPuYfILH2ub3ac+5h8gsfa5vdpqteccYLLdFEc+5h8gsfa5vdrsUWXXSiqoY79QUlPTTyNiZV0NQ+VrHuIDRI1zGloJIAcCeJGunSpPJcSIvFp/mCyvREXkRi8oJbjN3IOhFHMQR+wVPYyAMbtQAAApItAP2AqHKvixePqc33Cp7Gvi5avqkX3Aujg9DPj+F6mSREWSCLD4jl1pzvHKK+2Or7+tVY1zoKjk3x74Di0+S8Bw4tI4jzLMKAiLDeGNnGQ19jdWtZc6Cjjr6mJ7HNbFA9z2teXkbumsb+AOo04gcEGZRQ1n22YbfrVR3Oiuk0tvrbhDa6WpdQVLGT1Ev8mIy6MbzXeaQas/tK5S9wRF0aG+W+511wo6Stgqaq3yNiq4YpA51O9zQ9rXgfBJa5rtD5iD51R3kWJxfKrXmdpFys9Saui5aWn5Uxvj8uKR0cg0eAeD2OGumh01Go4rLKAiIqCIiAiw94y602C8WS119XyFfep5Kagi5N7uWkZE6VzdQCG6MY46uIHDTp0CzCgKez86YpVkdIfCR8x5VioVO7QPinWftw/7rFvwelp8Y9WVO+GxURFxmLF5V8WLx9Tm+4VPY18XLV9Ui+4FQ5V8WLx9Tm+4VPY18XLV9Ui+4F0cHoZ8fwvU5rzXSWyz11ZFCamWngklZC3pkLWkho+nTRaZ7nnG6zLsRxPaPds1yG7Xm70vf9TSMuLm2wGVp/QNpR5DWxk6DTyt5nEniFvJa/sewTA8ZyluQ2uwihuTJ5KmMQ1U4p45XhzXvZT7/ACTHEOcCWsHSUmNqNEYPd9e5n2U45ROv0l9vlVURUNLYLpzY+bkn1D5eWqQCY4mtBcd0bxIaADxC+bA3adl2zOut0dwu9fNiuZ1VBcqG3Xzk7nWUDImubDHXlsZe9j5QdXbheGaEgre03c+4BNZ5LWLAKeidcn3djKWrngdBVPbuvkheyQOh1GoLYy1p1PDiV1ndzbs6Nrkt0WPupKOSsbcDHR19TTkVAi5Iyh0cjS17mcHEEF3S7U8VhoyNN5XfKu441ime0N/zau2UUlmkZWy2y5mmu1DUsnIfVVTOHfDWBrmObq7dLHO3XA6rv7QY5L1kO3C3x5Be5bLUYRS3mnhZdZxHDIe+nEwje/RseIYw5rdA4agghxW1rp3N2zi8UdrpKnGY+87bSiip6WGqnhhMAeX8nIxkgbK3ec5xEgdqSSddVS1uzbG7hX3qsntodUXm2ts9c5s0jRLSN5TdiDQ4BunKycWgHyungNLoyNDS2efDtlmwCe136/wumvVohna671DmTxVMbXSRSNL9HxjcAax2oaNQAASqHY7abhtcfcc3vWX5FS11PkNZSw2W3XF1PR0cNNUujZTyQN8mQuawF5eC47/Dd4LbdXs7x6utGPWue379Dj89NU22LlpByEkDd2F2odq7dHmcSD59Vgq7YJgdxy9+Ty2EMvElRHWSyQVU8MU07CCyWSFjxG94LQd5zSdR0poyNB5LmeQjNaDNsXrMjbjbs0p7HPUXTIC6kqmOqxTTxQ24RlojDi8NkLmvBbroVbbH8Bopds22qqN0vrZWXlkXJtvNSI92aghcXFm/ulzS8hjiNWBrQ0gNGl5ce5w2dXa5VtfVY4H1FXVd/P3ayoYxlTvh5niY2QNilLhqZIw1x1OpOp1zr9lOLvz4ZqLa6LJS1rH1kFVNG2UNYWNMkTXiOQhri0F7SQOg8AkUzcecNnN5yDMbRspwutyq/UtuulTkk1bc4rlKLhWCjrnsgp++iTIAGv1Oh1LYwAQAV+syDI63ILVhHhffX2+37Q57CLtDWltXU0Ztkk5p5ZRxe5jnFu+fKG60ghzQ4b/rdheD3DFaLHJrJ/yqiq5q6lbHVzsmp55ZHySSRzteJGFzpXng4cHadGgXZtWxzDbHQ2CjoLJFS09irX3GgbHLIDHUuY9j5Xu3tZHFsjwTIXa669IGk0ZEtsOq6+35XtMxOputfeKDH7xAygnulS6pqGRTUcM5jdK8lzw1z3aFxJ0OmvBc/dQXe52LYpeKyz3Kqs9xbWW5kVbRP3JYt+vp2O0PRxa4gggggkEEEhZ+64RcbXc7pdcKntFmu16qGVF2qLtR1Fa2pdHE2KMtY2oiEZDGNHDgdOjXUrqOwTIcwoau0bQa6wX6wTiOTvS1W2poJOWjmjljcZDVyatDowd0AanTU6ag5Wm1hpXadlmRbCLttEobBfrtdYWYZHeqbnysfXOo6rvt1O+ZjpNSG7rg8s+DqzgAOC7Od3q9dznfra+y5Ne8xjuOM3muqaO+1zq0GejpmzRVMevGMOcSxzWaMIcNBqAvQVwwHH7tf6u9Vtsjq7hV2w2ed8znOZLRl5eYnRk7hBc46nTU66a6cFhML2HYPs+rKmrsdiZBU1FN3m6WpqJqpzafXXkWGZ79yPX+g3RvAcOCmjI0TT4nU2XMtgWTVWWX7Lbjdqiqq6k11cZaaSR9qnk3oItN2IcSGhmg0I11PFdLZOza7tNsONbQLfcQ2puNYyrqJJ8rmdRGnExE1Nzb3pybNGBzBpJvhwDi8nVbzxnucNneHX22Xiz4+aOutk0k9CRXVL46V0jHMeI43SFjGlr3DdDQ3oOmoBHbt+wPArTl3hLRWBtLde+XVoMVVO2nFQ4EOlFOH8kHnU6uDNePSpoyNgKd2gfFOs/bh/3WKiU7tA+KdZ+3D/usXrwOlp8Y9WVO+GxURFxmLF5V8WLx9Tm+4VPY18XLV9Ui+4FY1EEdVBJDK3fikaWOafOCNCFBw0t/xmnhtzbJNfKenY2KGso6iFrnsA0byjZXs0foOOhIPTw13R0OTzE0TRe03vtm3qyjbFmdRYTna/dTLr2qi9+nO1+6mXXtVF79b9DvR5o9yzNosJztfupl17VRe/Tna/dTLr2qi9+mh3o80e5Zm0WE52v3Uy69qovfpztfupl17VRe/TQ70eaPcszaLCc7X7qZde1UXv052v3Uy69qovfpod6PNHuWZtFhOdr91MuvaqL36x1Pm9fVZFXWOLFLq66UVLT1tRBy9INyGZ8zInb3LaHV1PMNAdRu8QNRq0O9Hmj3LKxFhOdr91MuvaqL36c7X7qZde1UXv00O9Hmj3LM2iwnO1+6mXXtVF79Odr91MuvaqL36aHejzR7lmbRYTna/dTLr2qi9+nO1+6mXXtVF79NDvR5o9yzNosJztfupl17VRe/Tna/dTLr2qi9+mh3o80e5Zm1O7QPinWftw/7rFz87X7qZde1UXv19Mtd3yp8VNXWl9ltrZY5ZjUzxvml3Hhwja2NzgAS0bzi7o1AB3tW50Ww6orqmLRt3xP5Ii03XiIi4rEREQEREBERAREQEREBQVkb/wB+uYnd6cdso13en/ibp59OP0anp6BrxvVr+xsA295o/dcCcbsY3t3gdKm68AdeJ49GnDUdOvANgIiICIiAiIgIiICIiAiIgIiICIiAiIgIiICIiAte2It8f+ajU73g1Y9Rujo76u2nHpPn4eb962EoCyB/j5zMkycn4OWTQEeQD3zdddD6ejX/APFBfoiICIiAiIgIiICIiAiIgIiICKbqdpOJ0c74ZsmtMc0Z3XsdWx6tPoI14H5lxeNLDutNo7bH7V6NXxp2xRPCVtOSpRS3jSw7rTaO2x+1PGlh3Wm0dtj9qavjdieErozkqUUt40sO602jtsftTxpYd1ptHbY/amr43YnhJozkqUUt40sO602jtsftTxpYd1ptHbY/amr43YnhJozko62tp7dRz1dXPFS0sEbpZp5nhjI2NGrnOceAAAJJPQtN4xtVwe4bfMm70zCwVMlfYrJSUvI3OB/fEoqrl+ij0ed945Rnkga+W3p3hpfT7TMKqYZIZslsssUjSx8b6uMtc0jQggniCv58bIu5mxzEe7Lra+qvNubgOPzi9WmsfVs5Od5dvU8IdrxdE/i7+7HRvBNXxuxPCTRnJ/TNFLeNLDutNo7bH7U8aWHdabR22P2pq+N2J4SaM5KlFLeNLDutNo7bH7U8aWHdabR22P2pq+N2J4SaM5KlFLeNLDutNo7bH7U8aWHdabR22P2pq+N2J4SaM5KlFLeNLDutNo7bH7U8aWHdabR22P2pq+N2J4SaM5KlFgLbn2M3irZS0OQWyrqpPgQw1cbnv+hoOp/cs+tVVFVE2riyWsIiLBBS20WpfHZaSma90bK6ugpZSxxaTG5/lt1BBGoBbqPMSqlSG0n9Ssn+L033ivTyaL41Piyp3uxBTxUsLIYY2QwxgNZHG0Na0DoAA6AuREXqYiIiAiIgIiICIiAiIgIiICIiAiIg69wt9PdKR9NVRNmhf0td5j5iD0gg8QRxB4hdvALlPdsMtFVUyGaofABJK7peW+SXH5zpr+9fC4NlvxAs390fvuWOLtwZ8Y9JXqVSIi5yCkNpP6lZP8XpvvFV6kNpP6lZP8XpvvFerkvTUsqd7uKS2p7SbXslwiuyW7aupqcxxMiEjIzLLI8MjZvPIa0Fzhq5xAaNSeAKrVDbaNm8m1PBJ7PS17bXc4amnuFBWvj5RkNTBK2WMub/AEmkt0I9BK9E3tsYtW0Hdh0UtvyoVNrtFRdbNYai/wAMFiySC509TFDoHxumjZrFJq5nAsIIJIJ0KsrJtmv1dlFNYblhrLPXXe0VF2sXKXZsravktzehnLY/0D/0sZO7yrQCdCdND0r5gm0PPNmOcY9kEGH22uu1okt9AbO+ocwSvY9rnyyPjBDCSzRrWOI0PFyz9Zs3udRtJ2d5C2ekFFjtpr6CrjL3co+SdtMGGMbuhaORdrqQeI0B46YbRpe3bYM9re47r8wyG1mol7x5U3O1ZB3lWzxGR4klaW0pbA9mjA1oDgQTxbpodhz53nUPdNuxSit1HW4vHj9LVubUXLknsa+ocySq0EDi6QbpYIi4AhgdvAuIGAh2E50zue8m2Uy1WPSUZoZaKy3Js07ZJGulc9pqWcmQzRpA8gv1IVzk2B5XT7ZLfm2My2eogmtDLJcqO7SSxObE2oMzZYXRsdvO8t43XaDo4qRcdKj7oA1mG2O4NsG7kdxyPwYlsRrP1aqZM9k5dLyfFrIo3za7g1aB0a6jb6824HjVDk3dQZZmFpqpqvELPCZt1kDxBz5JGKapdFq3y3NggaHbuvlTHz6rZ3j6xT/yMn//AFG7flVlE5jWeTd2rYLBdrw+KltNVYLRWPoqqd+R0sNye6N+5K+Cgd5cjGkO01c1zg0lrSCNazJO6Gkx3LK7EjjT6rLJ6mnZj9BHVnk7xTTAk1PK8n+iZFuS8rq125uDQu32ro4bstzrZ1cqu1WB+LXHC6u7SXKKa7xztuFHFPLys0DWNbuyaFz9xzntI3uIOmi+Mu2L5nkmdV2ewXyiosptNTHFi9G2eU0LKEfy8VV5GpdUbzt8ta7c3It0nd44/wDQ7eE57nd27oTPceqrZQzYxbBQBj+cdH0bJIZXh7IxB+lMjgN4OeNzQaF3QubANt2U7Q8SrclocBigtMLaxkJnvjGSVM8E7ot1odEGtjO64mR7m7pa4bpGjjkaPBcux/bNd8ptT7LPY8jp6GO609ZNM2ppX07Xt1g3WFsgc1/9Pc4hTrdgl7l7mao2cy3Cgju755p+VaZH0kodcHVTYpPJa4sewiN+g6HO01HTdo/MT7q+zXC05tVZBR0dslxWijuFTzLd4rtBPC/fDRHLGGjlN9hYWOAILm8dDqspctpG0d+z/KbtPgMOMzU1mmrrfLPe453iQN1DZWCHyHhurtBvtJZukjVSdb3OeSZvcMwdkj8dslvyLG4rK2mx3lXc3ywTGSB7d9jBK3V7iTpHputaAeLlsCyWLaTkdmutjzqXGI7dV2uahNXYnVD55ZXt3OVLZGtbGN0uO4C7iR5QA4o0usa8rdpWZu2B4LechtL4JLjW2WKevs2RGGofFM+n3ahx720/SPfo+AcN0uG/x4UmSd0ZX2uXLLlasLnvWHYnVPo7zeW3BkUrXxBrqgwU5aTKIg7yiXs1LXAa6LpM2R59dtjdkwq8zY62psdZZ+9auinn3aino54nvdIHR+RI5kQ0a3Ubx+EAuDJdiGdGiz3FMdulhgw7NK6orKurrhN3/QCqaBVsija3k5Q7yy0uczd3zrroFNozOQ7f7xT3zMKPHMMbkNHjFFTXKqrXXZtMJoJoDMOSaY3F0mjXaNOjTpxcCQFtXGMgpctxq03yh3jRXOkhrYN8aO5ORge3UenRwWtrTsbrrLdtpr6aekbb8itNDbbYx0jy+LkKSSA8r5OgGrm6FpdwB4DoVvsxxqqwvZtiePV0kMtbabTSUE8lOSY3SRQsY4tJAJaS06agHTzBZRfrFKuDZb8QLN/dH77lzrg2W/ECzf3R++5ZYvQT4x6SvUqkRFzUFIbSf1Kyf4vTfeKr1LbRKWSazUlSxj5G0VdBVStY0udybX+WQACToCXaDjwXp5NNsam+bKN7mRcdNUw1kDJ6eVk8LxvMkjcHNcPSCOlci9W5iIiICIiAiIgIiICIiAiIgIiICIiAuDZb8QLN/dH77l83C401qpXVFVM2GJvnPEk+YADiSTwAGpJIAXcwG2z2jDbRSVUZhqGQAyRE6ljj5RafnGun7lji7MGfGPSV6lAiIucgiIgnKvZvidfUST1OMWeeeQ7z5JKGIucfSTu8SuHxV4Z1Tsn2fF+FVKLfHKMaNkVzxlbzmlvFXhnVOyfZ8X4U8VeGdU7J9nxfhVSiusY3bnjJec0t4q8M6p2T7Pi/CnirwzqnZPs+L8KqUTWMbtzxkvOaW8VeGdU7J9nxfhTxV4Z1Tsn2fF+FVKJrGN254yXnNLeKvDOqdk+z4vwqHs+znFpNteW0b8ftT6KGwWeWKkdRxGOKR9Rcg97W6cC4MjBOg1EbeJ3eG4VAWRzvHzmTS7VgxyyEN8rge+bpqfRx4dHHhx8yaxjdueMl5zZbxV4Z1Tsn2fF+FPFXhnVOyfZ8X4VUomsY3bnjJec0t4q8M6p2T7Pi/CnirwzqnZPs+L8KqUTWMbtzxkvOaW8VeGdU7J9nxfhTxV4Z1Tsn2fF+FVKJrGN254yXnNLeKvDOqdk+z4vwp4q8M6p2T7Pi/CqlE1jG7c8ZLzmwNswLGrLVsqrfj1roqlnwZqejjY9v0OA1CzyItVVdVc3qm5e4iIsEEREBERAREQEREBERAWv7G0jb3mbuS0acbsgEuh8o983Xh6OGo6OPlcfMtgLX1jjI2+5o/k3gHGrGN8/BOlTdeA4dI148fOEGwUREBERAREQEREBERAREQEREBERAREQEREBERAWv7G0Db1mbtG6nG7INRvb36zdenzafRx6dfMrupbM6nlFO9kc5YRG+Rhe1rtOBLQQSNfNqPpC/n5sk7pnbRkndhVeIXHG8Zp7pO+mtN8MNLUhsFFRS1MjpYiZyA5zamXRzt5pJj0HTqH9CEREBERAREQEREBERAREQEREHDWVcdBST1Mx3YYWOkefQ0DU/6KFhlyDIqeKvN9qLJFUNEkVHQwQO5NhGoD3SxvLnaEa6AAHgOjU0+afE6+/UJ/8AbcsRY/5lt/1eP7oXQwIimia7RM3tti/qy3Q6PM9966Xjs1D+XTme+9dLx2ah/LrNot/Od2PLT7JdhOZ7710vHZqH8unM9966Xjs1D+XWbROc7seWn2LsJzPfeul47NQ/l05nvvXS8dmofy6zaJzndjy0+xdhOZ7710vHZqH8upm37HKa159dM2pb9cocpudLHRVdxbBR78sLNN1pHIbo+C3UgAndbqToFsFE5zux5afYuwnM9966Xjs1D+XTme+9dLx2ah/LrNonOd2PLT7F2E5nvvXS8dmofy6cz33rpeOzUP5dZtE5zux5afYuwnM9966Xjs1D+XTme+9dLx2ah/LrNonOd2PLT7F2E5nvvXS8dmofy6c5XjE9ysrLxNe7dvsjnZVwRMljDnBvKMdExoOhPFpHRqdRpoc2p3aF8Trl+y377VnRbEqiiqmLTs3RHpCxN5s2KiIuKxEREBERBhs0+J19+oT/AO25Yix/zLb/AKvH90LL5p8Tr79Qn/23LEWP+Zbf9Xj+6F0cHof5/C9TuouvcIxNQVMZqHUodE5vLsIDo9QfKBPQR0/uXg+5UlmwXZVnWGxU1oul5ZZKO8Oy+w1rqiK7UbLhEDNUsLjyc/HeJ1cHDUh2jdFKqrI98IvK23y+228bSMtjoK+mrXw7I78+RtPK2TcD5ICwnQ8NQCR6V07dsdw+XabsjpJbJFNS37Fq2pu8Msj3suUsbKQskqQXfpnAyvOr9eJB8w0aWQ9aovGeK2alyel2NY3dmyXCzQZfklsFNUTPcH0sDK0RQvOurmARsbukkFrQCCOCxlz2f2HGdl+1m/2yh7zvGKZryFiq45X79thbNSOEUGrv0cZM0urG6A751HRpNIe30Xh3ujLjR3Kv2i5pbYbBjV3xK50tFFc62pnN4qamMQO1gAlayGIteABuvDwHkga6rb+IbPsfzDuldrNzvNvius1uqLLJQtqfLjp5BRtdyrG9Afq1vlaagDQEanVpXmw3bjuQHIY695tlwtnelbNR7txg5Izcm7TlY+J3ondLXcNR5gssvF0Fj2fY7seyeG8YlaL5UUedXq0YvZ6xobEKiSqLI428QGRgMaXHoDIyfMs3ddnmIYNs72ebL7dQ43kE14uFTU1N0uc7m2xlXHByk8kkcL28o4hwbHCXABrW8RuAppD07k+YWrD47Y+61Bpxcq+C2UobG5/KVErt2NvAHTXjxOgGi+6/IDQZDabVzZcKkXBk7+/qeDepqbkw06TP18gv3tG8DqWu6NF4ppbZZL/sZxulyCS1ZBZbBtTFqZVnWSjhoHVOhY10j3kQEPaBvPd5O6CSOK21tIsdux3bHs3r8Jt1DHcGYrf6SgdQsaWvbBDB3tEN3gWsc9wA828U0usekkXjTCabF7Na9g2Q4jXiq2gZBcaVl8qI6x01XcIpKaR1w76aXEuEbxr5Q8hzWgaLq4LilrsWyfY/mlDTmDKajM6ehmugkeZpKaWvmgfAXE/yXJ6Dc+CNNdNU0h7VU7tC+J1y/Zb99qolO7Qvidcv2W/favVgdLT4x6sqd8NioiLjMRERAREQYbNPidffqE/+25Yix/zLb/q8f3QsxmTS/EL40DUmhnAH/TcsNYiHWS3kEEGnj0I/ZC6OD0P8/hep3XsbIxzHtDmOGha4agj0LAWXZ7i2NQV0Foxqz2qGvBFXHRUEULakEEHlA1o3+BPTr0lUCKomLdstwu0QTQ0OI2GihmppaOSOntkMbXwS6GWJwDRqx+63eaeDtBqDosu3HbUyroKptso21VvhdT0c4p2B9NE4NDo43aasadxmoGgO6PQFkESww9Nh1go5KSSnsdtgko6iarpnR0kbTBNLvcrIwgeS9++/ecOLt52uupSbDrBUUNwopbHbZaK4z99VtM+kjMdVN5J5SRpGj3+QzynanyW+gLMIgn7ls8xW83aa61+M2euuc0Bppa2poIpJpIi3dMbnlpcWkEjdJ00OiyFtx61WaoqZ7fbKOhnqWxtnlpoGRulEbdyMOLQC4NaA1uvQOA4LIIlhK3jZPhGRQiK64bj9ziFRNVhlZa4JWieUgyy6OYfLeQC53S4gakr4i2RYLT2F9jiwrHY7K+o77dbWWqAUzptA3lTHubu/oAN7TXQBVqJaBg3YLjb7bX252PWp1vuBaaykNFFyVTuta1vKM3dH6Na0DUHQNA8wXLQ4fYbWbYaKyW6kNrjkhoDBSRs70ZJpyjYtB5Adut1DdAdBr0LLolhgrXgmNWO91d5t2O2q33er174uFLRRR1E2p1O/I1oc7U8eJXLHh1ghttFbo7HbWW+iqG1dLSNpIxFTzNeXtkjZpo14eS4OABBJPSswiWBTu0L4nXL9lv32qiU9tBGuH3EDpLWAfOd9ui34HS0+MerKnfDYiIi4zEREQEREHy9jZGOY9ocxw0LXDUEehRhw+92od72a60QtzOEMFwpXyyQt/qiRsjd5o6BqNQBxJVqi3YeLVhf+fdb2RPMOYes7H2Gb3ycw5h6zsfYZvfK2RbtaxMo4QXRPMOYes7H2Gb3ycw5h6zsfYZvfK2RNaxMo4QXRPMOYes7H2Gb3ycw5h6zsfYZvfK2RNaxMo4QXRPMOYes7H2Gb3yn6Guy6tz284yKmzMdbbbQ3E1Ro5i2QVEtVGGAcrwLe9CddeO+OjTjtZa+sbwdvmaM3eIxuxne4cdam68OjXzec6ceGnHVrWJlHCC7t8w5h6zsfYZvfJzDmHrOx9hm98rZE1rEyjhBdE8w5h6zsfYZvfJzDmHrOx9hm98rZE1rEyjhBdE8w5h6zsfYZvfJzDmHrOx9hm98rZE1rEyjhBdE8w5h6zsfYZvfLmo8QulbVQvv1xpaqlhe2VtHQ0zoWve06tMjnSOLgCAQ0AcQNdRwVgik8qxJjZaP4guIiLyIIiICIiAiIgIiICIiAiIgLX9jcTt7zNvKEgY3Yzyep8n/ibrx00046en+j9GuwFr6xyk7fc0j04NxqxuB1Pnqbr5tdPN6P/wCaBsFERAREQEREBERAREQEREBERAREQEREBERAREQFr6xlvj9zQAN3vBqx6ka72nfN10182nTppx6dfMrusq46CjnqZd/koY3SP5ON0jtANTo1oJceHQASfMvMOEd2DslyTugLpDb8vfUuvtvs9ntsPNla3laxtTX77NHQjc/WINXO0HHp8k6B6kREQEREBERAREQEREBERAREQEREBERAUzku0fH8UqO9q+vBrdAe86ZjppgD0EtYCWg+Yu0HzrAbVs+msIis1slEdzqYuVlnHE00JJaHD+24hwbrwG64+YA6bhgZAHBg4uJc5xJLnOPS5xPEk+cniV9FyD4VrFPO402pndEb5Nkb23HbfLKD5NnvTx6RTxj/AFkBX54/bP6lvf8ABh96tTou58n5JlPE0vo2x4/bP6lvf8GH3q8qYDsexjC+6uvW0zmevfjhDqy1W1kEXKQVsvCQlu/uhjdXluh11cOjdW00T5PyTKeJpfRtjx+2f1Le/wCDD71PH7Z/Ut7/AIMPvVqdE+T8kyniaX0bbj292R7gH2q8wt103nU7HAf5ZCf/AKVdjWcWPLw8Wq4R1E0Y3pKd7XRzMHRq6NwDgNeGpGhXnZfm65k8VRDI+nqoXb0NRC7dkjd6Wn/UdBHAgjgtGL8F5PVT+nMxPGC8PU6KO2a5ycxtk8dUGMu1EWtqWRjRrmu13JGjzB267h5i1w46amxXx2NhV4Fc4dcWmAREWkEREBERAREQEREBERB5pymtdcs2ySpeSXd/OgGp6GxARgD0fBJ+klY9ZzaBaH2LPbvE5u7DXOFfTu/rNcAHjX0h7XfQHN9PGVvdymtNvfUwW2ru0jSAKWiMYldqdNRyj2N4dPFy/T+T1UzgUVU7rR6JVvd5YvKshp8Sxm63urDnU1upZKqRrPhOaxpcQPnOmiwPh/deoGTf5qD80uKpvM2aUlTYbphF/pLdcoX0tRNUvo+TZG9pDidyoc4cD0gErKcWJi1O/wAJ9kR+FbYskvWQ2SCutbKiguri17aO0XCB1v1YXNc+aaMRyt1G6S3d4uBAIXziW1rLa+z4Nf7vS2YWjJKtlvdTUTJRPBI9r9yTfc4tLS6Pizd1AcPKcrLB8JyTFJKSmrcxdebNRQd709I+3RxSloADDLMHEvLQNNQG69J1WPtux7m/C8Jx/nflPBq4Q13fHe2nfPJ7/kbu/wCRrv8ATqdNOheOmjlFomZn7b9n1n6/6w15tMzHJs92a3m8UsFqpcN5zgpoGyiR1bO2KujjMwcDuNBkYdGlpO7x1Xopaguewm5z2i5WC35eaLFquuFfHbZLa2V8DuXE7mNl3wdwvBIGmo16T57Gpzq509TLEzBcjqGseWiWJ1DuvAPwhrUg6Hp4gH5lng6dFU1YsTtiPrt23tbq27BXIowbQLqf/D/Jv81B+aVZb6p9bQwVElLNRSSMDnU1Ru8pGT/Rduuc3UfMSPnXtprird6Sit2WVrqHaRb2NJ3a2mnp3jXgdAJAdPSNw/5j6V6AWi9jlofcs3luG7rTWymcwv8A/el3d0D6GNeSP7bfTx3oviPjU0zyq0b4iL/7ws2dUCIi4KCIiAiIgIiICIiAiIgnM5wmmza1tgkk71rYHGSlq2t3jE/TQgjUbzD0ObqNeBBBDSNCX613HE5zDe6N9Fod1tU3V9NJ87ZNNP3O3XfMvTy/CA4EEag9IK7HIviWJyONC2lTl7SeLyeLxQOAIrqYg+cTN9qc70Py2n/it9q9QyY/a5nlz7bRvcelzoGE/wCi+fBq0eqqLs7PYuz8+w/254/0Wh5g53ofltP/ABW+1Od6H5bT/wAVvtXp/wAGrR6qouzs9ieDVo9VUXZ2exPnuH+3PH+i0PMHO9D8tp/4rfanO9D8tp/4rfavT/g1aPVVF2dnsTwatHqqi7Oz2J89w/254/0Wh5fN5t7emuph5uMzfas9jOL3fNJWNtdM6OkdpvXOpjIgYPS0cDIfQG8OjVzddV6GhsVtpn78NvpYnj+kyBoP/wBBd5aMX47M02wqLTnM3+1jYxOL4zRYjZ4rdQtcY2EufLIdZJnn4T3nzk/uAGgAAAAyyIvl6qqq6pqqm8yCIixBERAREQf/2Q==",
      "text/plain": [
       "<IPython.core.display.Image object>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "display(\n",
    "    Image(\n",
    "        app.get_graph().draw_mermaid_png(\n",
    "            draw_method=MermaidDrawMethod.API,\n",
    "        )\n",
    "    )\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Testing the Pipeline\n",
    "This cell runs a sample text through our pipeline and displays the results."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 22,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Classification: News\n",
      "\n",
      "Entities: ['OpenAI', 'GPT-4', 'GPT-3']\n",
      "\n",
      "Summary: OpenAI's upcoming GPT-4 model is a multimodal AI that aims for human-level performance, improved safety, and greater efficiency compared to GPT-3.\n"
     ]
    }
   ],
   "source": [
    "sample_text = \"\"\"\n",
    "OpenAI has announced the GPT-4 model, which is a large multimodal model that exhibits human-level performance on various professional benchmarks. It is developed to improve the alignment and safety of AI systems.\n",
    "additionally, the model is designed to be more efficient and scalable than its predecessor, GPT-3. The GPT-4 model is expected to be released in the coming months and will be available to the public for research and development purposes.\n",
    "\"\"\"\n",
    "\n",
    "state_input = {\"text\": sample_text}\n",
    "result = app.invoke(state_input)\n",
    "\n",
    "print(\"Classification:\", result[\"classification\"])\n",
    "print(\"\\nEntities:\", result[\"entities\"])\n",
    "print(\"\\nSummary:\", result[\"summary\"])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Conclusion\n",
    "\n",
    "In this tutorial, we've:\n",
    "- Explored LangGraph concepts\n",
    "- Built a text processing pipeline\n",
    "- Demonstrated LangGraph's use in data processing workflows\n",
    "- Visualized the workflow using Mermaid\n",
    "\n",
    "This example showcases how LangGraph can be used for tasks beyond conversational agents, providing a flexible framework for creating complex, graph-based workflows."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.12.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}


{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': ['# Introduction to LangGraph\n',
    '\n',
    'LangGraph is a framework for creating applications using graph-based workflows. Each node represents a function or computational step, and edges define the flow between these nodes based on certain conditions.\n',
    '\n',
    '## Key Features:\n',
    '- State Management\n',
    '- Flexible Routing\n',
    '- Persistence\n',
    '- Visualization']},
  {'cell_type': 'markdown',
   'metadata': {},
   'source': ['## Tutorial Overview: Text Analysis Pipeline\n',
    '\n',
    "In this tutorial, we'll demonstrate the power of LangGraph by building a multi-step text analysis pipeline. Our use case will focus on processing a given text through three key stages:\n",
    '\n',
    "1. **Text Classification**: We'll categorize the input text into predefined categories (e.g., News, Blog, Research, or Other).\n",
    "2. **Entity Extraction**: We'll identify and extract key entit

In [14]:
# prompt: state_input = {"text": "yahan apna text dalen"}

state_input = {"text": "im hafiz naveed uddin"}
print(state_input)


{'text': 'im hafiz naveed uddin'}


# ***DESCRIPTION***

Aapne LangGraph ka Text Analysis Pipeline Project successfully banaya hai. Is project mein aapne LangGraph framework ka use karke ek text analysis pipeline develop ki hai. Is pipeline ka kaam yeh hai ke yeh kisi bhi text ko analyze kar ke usmein se important information nikal sakti hai, jaise:

Text Classification: Text ko alag-alag categories mein divide karna (jaise positive/negative sentiment analysis).
Entity Extraction: Text mein se important entities (jaise logon ke naam, jagah, dates) ko identify karna.
Summary Generation: Text ka short summary generate karna.
Is Project Ka Faida:
Business Automation: Yeh pipeline aapko automated text analysis mein madad karegi. Aap isko customer feedback, emails, ya social media posts ko automatically analyze karne ke liye use kar sakte hain. Isse time aur resources ki bachat hoti hai.

Data Insights: Text analysis kar ke aap apne data ke important insights jaan sakte hain. Jaise ke aap kis tarah ka feedback receive kar rahe hain ya customers kin topics par zyada focus kar rahe hain.

NLP (Natural Language Processing) Tasks: Yeh project Natural Language Processing ke kaafi important tasks automate karta hai, jaise:

Sentiment Analysis: Aap jaan sakte hain ke kisi text ka mood ya tone kya hai (positive, negative, ya neutral).
Entity Recognition: Text mein important information, jaise naam, locations ya dates nikalne ke liye yeh kaafi useful hai.
Time-saving for Large Text Analysis: Agar aapko bohat zyada text data analyze karna ho, to yeh project aapka kaam asaan kar dega kyunki yeh automatically data ko process karke key points nikal deta hai.

Practical Uses:
Customer Support: Customer reviews aur complaints ko analyze kar ke jaldi se unka solution find karna.
Content Summarization: Badi-badi reports ya articles ko summarize karna.
Market Research: Social media ya surveys ke through logon ka opinion jaan na aur unko analyze karna.
Aap is project ko modify karke aur zyada features bhi add kar sakte hain ya apne specific business needs ke liye use kar sakte hain.

# ***new programe in advance***

In [15]:
!pip install transformers langchain spacy
!pip install torch torchvision torchaudio
!pip install summarizer
!python -m spacy download en_core_web_sm  # Spacy model for entity extraction


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summarizer: filename=summarizer-0.0.7-py2.py3-none-any.whl size=284210 sha256=ba0697cf659194ada641ab6dd83d3b7de531c7a6c9c368ff651254ed7e5bec89
  Stored in directory: /root/.cache/pip/wheels/20/bb/2d/1fe057c2f729818a5f28c312c3667e8b9d5cfd4af4a39895e7
Successfully built summarizer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 57.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [16]:
from transformers import pipeline

# Pre-trained model for sentiment analysis
classifier = pipeline('sentiment-analysis')

# Text input
text = "I absolutely love this product! It's amazing."

# Perform sentiment analysis
result = classifier(text)
print("Sentiment Classification:", result)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Sentiment Classification: [{'label': 'POSITIVE', 'score': 0.9998855590820312}]


In [17]:
import spacy

# Load spacy's English model
nlp = spacy.load("en_core_web_sm")

# Text input
doc = nlp("Apple is looking at buying U.K. startup for $1 billion by the end of 2024.")

# Extract entities
for ent in doc.ents:
    print(ent.text, ent.label_)


Apple ORG
U.K. GPE
$1 billion MONEY
the end of 2024 DATE


In [18]:
from transformers import pipeline

# Pre-trained model for summarization
summarizer = pipeline("summarization")

# Text input (long text)
text = """
Artificial intelligence (AI) is intelligence demonstrated by machines, in contrast to the natural intelligence displayed by humans and animals. Leading AI textbooks define the field as the study of "intelligent agents": any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals. Colloquially, the term "artificial intelligence" is often used to describe machines (or computers) that mimic "cognitive" functions that humans associate with the human mind, such as "learning" and "problem-solving".
"""

# Generate summary
summary = summarizer(text, max_length=50, min_length=25, do_sample=False)
print("Summary:", summary)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Summary: [{'summary_text': ' Leading AI textbooks define the field as the study of "intelligent agents" Colloquially, the term "artificial intelligence" is often used to describe machines (or computers) that mimic "cognitive" functions that humans associate with'}]


In [19]:
from transformers import pipeline
import spacy

# Initialize models
classifier = pipeline('sentiment-analysis')
summarizer = pipeline("summarization")
nlp = spacy.load("en_core_web_sm")

# Function to process text
def process_text(text):
    print("Processing Text...")

    # Step 1: Text Classification
    sentiment = classifier(text)
    print("Sentiment Classification:", sentiment)

    # Step 2: Entity Extraction
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    print("Entities:", entities)

    # Step 3: Summary Generation
    summary = summarizer(text, max_length=50, min_length=25, do_sample=False)
    print("Summary:", summary)

# Example Text Input
text = """
Google, headquartered in Mountain View, unveiled the new Pixel at a
press event in New York City. The new Pixel features a 12.2 MP camera
and a Snapdragon 855 processor. Google is also planning to launch
a new AI research lab in Paris next year.
"""

# Run the comprehensive analysis
process_text(text)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Processing Text...
Sentiment Classification: [{'label': 'POSITIVE', 'score': 0.970305323600769}]
Entities: [('Google', 'ORG'), ('Mountain View', 'GPE'), ('Pixel', 'PERSON'), ('New York City', 'GPE'), ('Pixel', 'ORG'), ('12.2', 'CARDINAL'), ('Snapdragon', 'ORG'), ('855', 'CARDINAL'), ('Google', 'ORG'), ('AI', 'ORG'), ('Paris', 'GPE'), ('next year', 'DATE')]
Summary: [{'summary_text': ' The new Pixel features a 12.2 MP camera and a Snapdragon 855 processor . Google is also planning to launch a new AI research lab in Paris next year .'}]


In [20]:
# Example text for testing
text = """
Apple is looking at buying U.K. startup for $1 billion by the end of 2024.
This acquisition will allow Apple to expand its AI capabilities.
"""

# Run the comprehensive analysis
process_text(text)


Your max_length is set to 50, but your input_length is only 38. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)


Processing Text...
Sentiment Classification: [{'label': 'POSITIVE', 'score': 0.9973506927490234}]
Entities: [('Apple', 'ORG'), ('U.K.', 'GPE'), ('$1 billion', 'MONEY'), ('the end of 2024', 'DATE'), ('Apple', 'ORG'), ('AI', 'ORG')]
Summary: [{'summary_text': ' Apple is looking at buying U.K. startup for $1 billion by the end of 2024 . This acquisition will allow Apple to expand its AI capabilities .'}]


In [23]:
!pip install rake-nltk


In [27]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [28]:
from rake_nltk import Rake

def keyword_extraction(text):
    r = Rake()  # Initialize Rake object
    r.extract_keywords_from_text(text)  # Extract keywords
    keywords = r.get_ranked_phrases()  # Get the keywords
    print("Keywords:", keywords)

# Example text for testing
text = "The new Tesla Model S is one of the most advanced electric cars, with features like autopilot and self-driving. Elon Musk aims to revolutionize the car industry with this innovation."

# Run keyword extraction
keyword_extraction(text)


Keywords: ['new tesla model', 'features like autopilot', 'elon musk aims', 'advanced electric cars', 'car industry', 'self', 'revolutionize', 'one', 'innovation', 'driving']


In [29]:
# Full system with all components (classification, entity extraction, summary, and keywords)
def process_text_with_keywords(text):
    print("Processing Text with Keywords...")

    # Step 1: Text Classification
    sentiment = classifier(text)
    print("Sentiment Classification:", sentiment)

    # Step 2: Entity Extraction
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    print("Entities:", entities)

    # Step 3: Summary Generation
    summary = summarizer(text, max_length=50, min_length=25, do_sample=False)
    print("Summary:", summary)

    # Step 4: Keyword Extraction
    keyword_extraction(text)

# Example text for testing the full system
text = """
The new Tesla Model S is one of the most advanced electric cars, with features like autopilot and self-driving. Elon Musk aims to revolutionize the car industry with this innovation.
"""
process_text_with_keywords(text)


Processing Text with Keywords...


Your max_length is set to 50, but your input_length is only 42. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)


Sentiment Classification: [{'label': 'POSITIVE', 'score': 0.999646782875061}]
Entities: [('Tesla Model S', 'PERSON'), ('Elon Musk', 'PERSON')]
Summary: [{'summary_text': ' Tesla Model S is one of the most advanced electric cars, with features like autopilot and self-driving . Elon Musk aims to revolutionize the car industry with this innovation .'}]
Keywords: ['new tesla model', 'features like autopilot', 'elon musk aims', 'advanced electric cars', 'car industry', 'self', 'revolutionize', 'one', 'innovation', 'driving']
